In [4]:
library(tidyverse)
library(magrittr)
library(reshape2)
library(ggplot2)
library(scales)
library(ggrepel)
library(readxl)
library(corrplot)
library(purrr)
library(rlang)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

The following object is masked from 'package:tidyr':

    extract


Attaching package: 'reshape2'

The following object is masked from 'package:tidyr':

    smiths


Attaching package: 'scales'

The following object is masked from 'package:purrr':

    discard

The following object is masked from 'package:readr':

    col_factor

corrplot 0.84 loaded

Attaching package: 'rlang'

The following object is masked from 'package:magrittr':

    set_names

The following objects are masked from 'package

# EDA

# Importing The Data

In [22]:
input_path <- "D:/OneDrive - UTS/36103/AT2B/EDA"
output_path <- "D:/OneDrive - UTS/36103/AT2B/data2"

In [3]:
input_list <- list.files(input_path)
input_list

[1] "age.csv"                 "country_of_birth.csv"   
[3] "crime.csv"               "industry_employment.csv"
[5] "labour.csv"              "pop_dens.csv"           
[7] "seifa.csv"               "transport.csv"

In [4]:
tbl_names <- map(input_list, function(x) strsplit(x, "\\.")[[1]][[1]])
tbl_names

[[1]]
[1] "age"

[[2]]
[1] "country_of_birth"

[[3]]
[1] "crime"

[[4]]
[1] "industry_employment"

[[5]]
[1] "labour"

[[6]]
[1] "pop_dens"

[[7]]
[1] "seifa"

[[8]]
[1] "transport"

In [5]:
input_list <- map(input_list, ~paste(input_path, .x, sep = "/"))
input_list

[[1]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/age.csv"

[[2]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/country_of_birth.csv"

[[3]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/crime.csv"

[[4]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/industry_employment.csv"

[[5]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/labour.csv"

[[6]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/pop_dens.csv"

[[7]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/seifa.csv"

[[8]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/transport.csv"

### Column Definitions

In [6]:
cols_list <- list(age = cols(Age = readr::col_factor()))

In [7]:
cols_list <- list(age = cols(Age = readr::col_factor(),
                             Sex = readr::col_factor(),
                             `Labour force status` = readr::col_factor(),
                             LGA = readr::col_factor(),
                             Value = readr::col_integer()),
                  country_of_birth = cols(birthplace_of_person = readr::col_factor(),
                                          LGA = readr::col_factor(),
                                          Value = readr::col_integer(),
                                          year_of_arrival_in_australia = readr::col_factor()),
                  crime = cols(LGA = readr::col_factor(),
                               Offence = readr::col_factor(),
                               Crim_Count = readr::col_integer()),
                  industry_employment = cols(LGA = readr::col_factor(),
                                             Value = readr::col_integer(),
                                             Age = readr::col_factor(),
                                             Sex = readr::col_factor(),
                                             `Industry of Employment` = readr::col_factor()),
                  labour = cols(`Labour force status` = readr::col_factor(),
                                LGA = readr::col_factor(),
                                Value = readr::col_integer(),
                                Age = readr::col_factor(),
                                Sex = readr::col_factor()),
                  pop_dens = cols(LGA = readr::col_factor(),
                                  Pop_Dens_km2 = readr::col_double()),
                  seifa = cols(LGA = readr::col_factor(),
                               Value = readr::col_integer()),
                  seifa = cols(LGA = readr::col_factor(),
                               route_type = readr::col_factor(),
                               num_service = readr::col_double()))

In [8]:
input_data <- map2(input_list, cols_list, ~readr::read_csv(.x, col_types = .y))

Warning message:
"The following named parsers don't match the column names: Labour force status"Warning message:
"The following named parsers don't match the column names: Value"Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"Missing column names filled in: 'X1' [1]"

In [9]:
names(input_data) = tbl_names
names(input_data)

[1] "age"                 "country_of_birth"    "crime"              
[4] "industry_employment" "labour"              "pop_dens"           
[7] "seifa"               "transport"

In [10]:
map(input_data, head)

$age
# A tibble: 6 x 4
  Sex     Age     LGA                          Value
  <fct>   <fct>   <fct>                        <int>
1 Females 95 - 99 Lismore                         65
2 Females 95 - 99 Queanbeyan-Palerang Regional    33
3 Females 95 - 99 Mid-Coast                      173
4 Females 95 - 99 Glen Innes Severn               16
5 Females 95 - 99 Lockhart                        11
6 Females 95 - 99 Lane Cove                       49

$country_of_birth
# A tibble: 6 x 7
  birthplace_of_pe~ census_year lga_2016 LGA   value yarrp_2016 year_of_arrival~
  <fct>                   <dbl>    <dbl> <fct> <dbl> <chr>      <fct>           
1 Oceania and Anta~        2016    10050 Albu~     0 1          Arrived 1900 - ~
2 Oceania and Anta~        2016    10050 Albu~     0 2          Arrived 1946 - ~
3 Oceania and Anta~        2016    10050 Albu~     0 3          Arrived 1956 - ~
4 Oceania and Anta~        2016    10050 Albu~     0 4          Arrived 1966 - ~
5 Oceania and Anta~        2016    10050 Albu~     0 5          Arrived 1976 - ~
6 Oceania and Anta~        2016    10050 Albu~     0 6          Arrived 1986 - ~

$crime
# A tibble: 6 x 4
     X1 LGA    Offence                               Crim_Count
  <dbl> <fct>  <fct>                                      <int>
1     1 Albury Murder *                                       2
2     2 Albury Attempted murder                               1
3     3 Albury Murder accessory, conspiracy                   0
4     4 Albury Manslaughter *                                 0
5     5 Albury Domestic violence related assault            284
6     6 Albury Non-domestic violence related assault        296

$industry_employment
# A tibble: 6 x 7
  Age    Sex    `Industry of Employment` LGA_2016 LGA        `Census year` Value
  <fct>  <fct>  <fct>                       <dbl> <fct>              <dbl> <int>
1 15 - ~ Males  Agriculture, Forestry a~    10050 Albury              2016     6
2 15 - ~ Femal~ Agriculture, Forestry a~    10050 Albury              2016     4
3 15 - ~ Perso~ Agriculture, Forestry a~    10050 Albury              2016     6
4 15 - ~ Males  Agriculture, Forestry a~    10130 Armidale ~          2016    34
5 15 - ~ Femal~ Agriculture, Forestry a~    10130 Armidale ~          2016     9
6 15 - ~ Perso~ Agriculture, Forestry a~    10130 Armidale ~          2016    43

$labour
# A tibble: 6 x 7
  Age     Sex     `Labour force status` LGA_2016 LGA         `Census year` Value
  <fct>   <fct>   <fct>                    <dbl> <fct>               <dbl> <int>
1 15 - 19 Males   Employed, worked ful~    10050 Albury               2016   236
2 15 - 19 Females Employed, worked ful~    10050 Albury               2016   113
3 15 - 19 Persons Employed, worked ful~    10050 Albury               2016   348
4 15 - 19 Males   Employed, worked ful~    10130 Armidale R~          2016   105
5 15 - 19 Females Employed, worked ful~    10130 Armidale R~          2016    58
6 15 - 19 Persons Employed, worked ful~    10130 Armidale R~          2016   160

$pop_dens
# A tibble: 6 x 3
     X1 LGA               Pop_Dens_km2
  <dbl> <fct>                    <dbl>
1     1 Albury                   170. 
2     2 Armidale Regional          3.5
3     3 Ballina                   88.7
4     4 Balranald                  0.1
5     5 Bathurst Regional         11.1
6     6 Bega Valley                5.4

$seifa
# A tibble: 6 x 3
     X1 LGA               Value
  <dbl> <fct>             <int>
1     1 Albury              964
2     2 Armidale Dumaresq   954
3     3 Ashfield            962
4     4 Auburn              931
5     5 Ballina             986
6     6 Balranald           969

$transport
# A tibble: 6 x 3
  LGA               route_type num_service
  <fct>             <fct>            <dbl>
1 Albury            106                131
2 Albury            204                 14
3 Albury            700               1624
4 Armidale Regional 106                 62
5 Armidale Regional 204                 10
6 Armidale Re

In [11]:
for(i in 1:length(input_data)){
    print(names(input_data)[[i]])
    print(glimpse(input_data[[i]]))
}

[1] "age"
Observations: 5,808
Variables: 4
$ Sex   <fct> Females, Females, Females, Females, Females, Females, Females...
$ Age   <fct> 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99...
$ LGA   <fct> Lismore, Queanbeyan-Palerang Regional, Mid-Coast, Glen Innes ...
$ Value <int> 65, 33, 173, 16, 11, 49, 26, 451, 24, 0, 4, 39, 72, 29, 9, 51...
# A tibble: 5,808 x 4
   Sex     Age     LGA                          Value
   <fct>   <fct>   <fct>                        <int>
 1 Females 95 - 99 Lismore                         65
 2 Females 95 - 99 Queanbeyan-Palerang Regional    33
 3 Females 95 - 99 Mid-Coast                      173
 4 Females 95 - 99 Glen Innes Severn               16
 5 Females 95 - 99 Lockhart                        11
 6 Females 95 - 99 Lane Cove                       49
 7 Females 95 - 99 Nambucca                        26
 8 Females 95 - 99 Northern Beaches               451
 9 Females 95 - 99 Inverell                        24
10 Females 95 - 99 Hay   

## `tolower`

Make the `LGA` values all lower case in case of inconsistent capitalisation, e.g. "Ku-ring-gai" vs "Ku-Ring-Gai".

In [29]:
for(i in 1:length(input_data)){
    input_data[[i]]$LGA <- tolower(input_data[[i]]$LGA)
}

## Reducing Size

The `country_of_birth` data set contains many zero values. The following step removes these rows.

### Remove Zero Values

In [30]:
head(input_data$country_of_birth)

birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,albury,87,7,Arrived 1996 - 2005


In [31]:
input_data$country_of_birth %>%
    filter(value == 0) %>%
    dim

[1] 0 7

In [32]:
input_data$country_of_birth %<>%
    filter(value != 0)

head(input_data$country_of_birth)
glimpse(input_data$country_of_birth)

birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,albury,87,7,Arrived 1996 - 2005


Observations: 108,163
Variables: 7
$ birthplace_of_person         <fct> New Zealand, New Zealand, New Zealand,...
$ census_year                  <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 20...
$ lga_2016                     <dbl> 10050, 10050, 10050, 10050, 10050, 100...
$ LGA                          <chr> "albury", "albury", "albury", "albury"...
$ value                        <dbl> 7, 22, 63, 113, 92, 87, 6, 24, 7, 12, ...
$ yarrp_2016                   <chr> "2", "3", "4", "5", "6", "7", "9", "Z"...
$ year_of_arrival_in_australia <fct> Arrived 1946 - 1955, Arrived 1956 - 19...


# Joining the Data Sets Together

## Find the Data Set With The Most Complete List of LGAs

In [33]:
input_data %>% map(function(tbl) tbl['LGA'] %>% unique %>% dim %>% extract(1))

$age
[1] 132

$country_of_birth
[1] 132

$crime
[1] 130

$industry_employment
[1] 132

$labour
[1] 132

$pop_dens
[1] 129

$seifa
[1] 556

$transport
[1] 128

Seems like `seifa` has the most number of LGAs, but we know that there are only about 130 LGAs in NSW. So maybe the LGAs in the `seifa` data includes non-NSW LGAs too. A glance of the tail of the `seifa` data shows that there are indeed non-NSW areas, e.g. `Unincorporated ACT`.

We will not use the `seifa` data as the starting point, instead `country_of_birth` will be used for `left_join`ing the other data sets, i.e. LGAs that are not in the `country_of_birth` data set will not be joined.

In [34]:
input_data$seifa %>% 
    tail

X1,LGA,Value
559,tiwi islands,589
560,victoria-daly,613
561,wagait,1011
562,west arnhem,640
563,unincorporated nt,966
564,unincorporated act,1051


# Add Population Per LGA

This population table is to be used to calculate the crime rate, i.e. crime count divided by the number of males and females for that LGA.

In [35]:
head(input_data$age)
glimpse(input_data$age)

Sex,Age,LGA,Value,pop_total
Females,95 - 99,lismore,65,43140
Females,95 - 99,queanbeyan-palerang regional,33,56023
Females,95 - 99,mid-coast,173,90301
Females,95 - 99,glen innes severn,16,8828
Females,95 - 99,lockhart,11,3135
Females,95 - 99,lane cove,49,36053


Observations: 5,808
Variables: 5
Groups: LGA [132]
$ Sex       <fct> Females, Females, Females, Females, Females, Females, Fem...
$ Age       <fct> 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 ...
$ LGA       <chr> "lismore", "queanbeyan-palerang regional", "mid-coast", "...
$ Value     <int> 65, 33, 173, 16, 11, 49, 26, 451, 24, 0, 4, 39, 72, 29, 9...
$ pop_total <int> 43140, 56023, 90301, 8828, 3135, 36053, 19195, 252898, 16...


In [36]:
input_data$age %<>%
    group_by(LGA) %>%
    mutate(pop_total = sum(Value))

head(input_data$age)
glimpse(input_data$age)

Sex,Age,LGA,Value,pop_total
Females,95 - 99,lismore,65,43140
Females,95 - 99,queanbeyan-palerang regional,33,56023
Females,95 - 99,mid-coast,173,90301
Females,95 - 99,glen innes severn,16,8828
Females,95 - 99,lockhart,11,3135
Females,95 - 99,lane cove,49,36053


Observations: 5,808
Variables: 5
Groups: LGA [132]
$ Sex       <fct> Females, Females, Females, Females, Females, Females, Fem...
$ Age       <fct> 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 ...
$ LGA       <chr> "lismore", "queanbeyan-palerang regional", "mid-coast", "...
$ Value     <int> 65, 33, 173, 16, 11, 49, 26, 451, 24, 0, 4, 39, 72, 29, 9...
$ pop_total <int> 43140, 56023, 90301, 8828, 3135, 36053, 19195, 252898, 16...


In [37]:
pop_agg <- input_data$age %>%
    group_by(LGA) %>%
    summarise(pop_total = sum(Value))

head(pop_agg)
glimpse(pop_agg)

LGA,pop_total
albury,51081
armidale regional,29429
ballina,41774
balranald,2292
bathurst regional,41297
bega valley,33229


Observations: 132
Variables: 2
$ LGA       <chr> "albury", "armidale regional", "ballina", "balranald", "b...
$ pop_total <int> 51081, 29429, 41774, 2292, 41297, 33229, 12656, 8456, 336...


# Joining `crime` and `age`

In [43]:
head(input_data$crime)
head(input_data$age)

LGA,Offence,Crim_Count,crime_tot_per_lga,pop_total,crime_rate_per_lga
albury,Murder *,2,6233,51081,0.1220219
albury,Attempted murder,1,6233,51081,0.1220219
albury,"Murder accessory, conspiracy",0,6233,51081,0.1220219
albury,Manslaughter *,0,6233,51081,0.1220219
albury,Domestic violence related assault,284,6233,51081,0.1220219
albury,Non-domestic violence related assault,296,6233,51081,0.1220219


Sex,Age,LGA,Value,pop_total
Females,95 - 99,lismore,65,43140
Females,95 - 99,queanbeyan-palerang regional,33,56023
Females,95 - 99,mid-coast,173,90301
Females,95 - 99,glen innes severn,16,8828
Females,95 - 99,lockhart,11,3135
Females,95 - 99,lane cove,49,36053


In [47]:
joined <- input_data$crime %>%
    group_by(LGA) %>%
    mutate(crime_tot_per_lga = sum(Crim_Count)) %>%
    ungroup %>%
    select(-pop_total) %>%
    left_join(input_data$age, by = "LGA") %>%
    mutate(crime_rate_per_lga = crime_tot_per_lga / pop_total) %>%
    rename(pop_sex_age = Value)

head(joined)
glimpse(joined)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop_sex_age,pop_total
albury,Murder *,2,6233,0.1220219,Females,95 - 99,78,51081
albury,Murder *,2,6233,0.1220219,Males,95 - 99,21,51081
albury,Murder *,2,6233,0.1220219,Females,90 - 94,271,51081
albury,Murder *,2,6233,0.1220219,Males,90 - 94,117,51081
albury,Murder *,2,6233,0.1220219,Females,85 - 89,484,51081
albury,Murder *,2,6233,0.1220219,Males,85 - 89,351,51081


Observations: 341,310
Variables: 9
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ Offence            <fct> Murder *, Murder *, Murder *, Murder *, Murder *...
$ Crim_Count         <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ Sex                <fct> Females, Males, Females, Males, Females, Males, ...
$ Age                <fct> 95 - 99, 95 - 99, 90 - 94, 90 - 94, 85 - 89, 85 ...
$ pop_sex_age        <int> 78, 21, 271, 117, 484, 351, 734, 501, 924, 746, ...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...


In [48]:
out_filename = "joined_crime_age.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Adding Crime Rate 

The crime rate to the crime data set. 

In [51]:
input_data$crime %<>%
    group_by(LGA) %>%
    mutate(crime_tot_per_lga = sum(Crim_Count)) %>% 
    ungroup %>%
    select(-pop_total) %>%
    left_join(pop_agg, by = "LGA") %>%
    mutate(crime_rate_per_lga = crime_tot_per_lga / pop_total)

head(input_data$crime)
glimpse(input_data$crime)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


Observations: 8,060
Variables: 6
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ Offence            <fct> "Murder *", "Attempted murder", "Murder accessor...
$ Crim_Count         <int> 2, 1, 0, 0, 284, 296, 25, 47, 68, 2, 12, 3, 6, 0...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...


# Joining `crime` and `country_of_birth`

In [52]:
head(input_data$crime)
head(input_data$country_of_birth)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,albury,87,7,Arrived 1996 - 2005


In [53]:
joined <- input_data$crime %>%
    left_join(input_data$country_of_birth %>% select(-lga_2016, -census_year, -yarrp_2016)) %>%
    rename(pop_count = value)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,birthplace_of_person,pop_count,year_of_arrival_in_australia
albury,Murder *,2,6233,0.1220219,51081,New Zealand,7,Arrived 1946 - 1955
albury,Murder *,2,6233,0.1220219,51081,New Zealand,22,Arrived 1956 - 1965
albury,Murder *,2,6233,0.1220219,51081,New Zealand,63,Arrived 1966 - 1975
albury,Murder *,2,6233,0.1220219,51081,New Zealand,113,Arrived 1976 - 1985
albury,Murder *,2,6233,0.1220219,51081,New Zealand,92,Arrived 1986 - 1995
albury,Murder *,2,6233,0.1220219,51081,New Zealand,87,Arrived 1996 - 2005


Observations: 6,382,714
Variables: 9
$ LGA                          <chr> "albury", "albury", "albury", "albury"...
$ Offence                      <fct> Murder *, Murder *, Murder *, Murder *...
$ Crim_Count                   <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
$ crime_tot_per_lga            <int> 6233, 6233, 6233, 6233, 6233, 6233, 62...
$ crime_rate_per_lga           <dbl> 0.1220219, 0.1220219, 0.1220219, 0.122...
$ pop_total                    <int> 51081, 51081, 51081, 51081, 51081, 510...
$ birthplace_of_person         <fct> New Zealand, New Zealand, New Zealand,...
$ pop_count                    <dbl> 7, 22, 63, 113, 92, 87, 6, 24, 3, 9, 6...
$ year_of_arrival_in_australia <fct> Arrived 1946 - 1955, Arrived 1956 - 19...


In [54]:
out_filename = "joined_crime_countryofbirth.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `industry_employment`

In [55]:
head(input_data$crime)
head(input_data$industry_employment)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


Age,Sex,Industry of Employment,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Agriculture, Forestry and Fishing",10050,albury,2016,6
15 - 19,Females,"Agriculture, Forestry and Fishing",10050,albury,2016,4
15 - 19,Persons,"Agriculture, Forestry and Fishing",10050,albury,2016,6
15 - 19,Males,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,34
15 - 19,Females,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,9
15 - 19,Persons,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,43


In [56]:
joined <- input_data$crime %>%
    left_join(input_data$industry_employment %>% select(-LGA_2016, -`Census year`)) %>%
    rename(pop_count = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,Age,Sex,Industry of Employment,pop_count
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Males,"Agriculture, Forestry and Fishing",6
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Females,"Agriculture, Forestry and Fishing",4
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Persons,"Agriculture, Forestry and Fishing",6
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Males,Mining,0
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Females,Mining,0
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Persons,Mining,0


Observations: 4,650,310
Variables: 10
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ Offence                  <fct> Murder *, Murder *, Murder *, Murder *, Mu...
$ Crim_Count               <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ Age                      <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 1...
$ Sex                      <fct> Males, Females, Persons, Males, Females, P...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ pop_count                <int> 6, 4, 6, 0, 0, 0, 49, 44, 90, 0, 0, 0, 121...


In [57]:
out_filename = "joined_crime_employment.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `labour`

In [58]:
head(input_data$crime)
head(input_data$labour)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


Age,Sex,Labour force status,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Employed, worked full-time",10050,albury,2016,236
15 - 19,Females,"Employed, worked full-time",10050,albury,2016,113
15 - 19,Persons,"Employed, worked full-time",10050,albury,2016,348
15 - 19,Males,"Employed, worked full-time",10130,armidale regional,2016,105
15 - 19,Females,"Employed, worked full-time",10130,armidale regional,2016,58
15 - 19,Persons,"Employed, worked full-time",10130,armidale regional,2016,160


In [59]:
joined <- input_data$crime %>%
    left_join(input_data$labour %>% select(-LGA_2016, -`Census year`)) %>%
    rename(pop_count = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,Age,Sex,Labour force status,pop_count
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Males,"Employed, worked full-time",236
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Females,"Employed, worked full-time",113
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Persons,"Employed, worked full-time",348
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Males,"Employed, worked part-time",431
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Females,"Employed, worked part-time",641
albury,Murder *,2,6233,0.1220219,51081,15 - 19,Persons,"Employed, worked part-time",1074


Observations: 2,511,310
Variables: 10
$ LGA                   <chr> "albury", "albury", "albury", "albury", "albu...
$ Offence               <fct> Murder *, Murder *, Murder *, Murder *, Murde...
$ Crim_Count            <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ crime_tot_per_lga     <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 623...
$ crime_rate_per_lga    <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0...
$ pop_total             <int> 51081, 51081, 51081, 51081, 51081, 51081, 510...
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ pop_count             <int> 236, 113, 348, 431, 641, 1074, 27, 26, 59, 22...


In [60]:
out_filename = "joined_crime_labour.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `industry_employment`

In [61]:
head(input_data$crime)
head(input_data$pop_dens)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


X1,LGA,Pop_Dens_km2
1,albury,170.5
2,armidale regional,3.5
3,ballina,88.7
4,balranald,0.1
5,bathurst regional,11.1
6,bega valley,5.4


In [62]:
joined <- input_data$crime %>%
    left_join(input_data$pop_dens %>% select(-X1))

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,Pop_Dens_km2
albury,Murder *,2,6233,0.1220219,51081,170.5
albury,Attempted murder,1,6233,0.1220219,51081,170.5
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081,170.5
albury,Manslaughter *,0,6233,0.1220219,51081,170.5
albury,Domestic violence related assault,284,6233,0.1220219,51081,170.5
albury,Non-domestic violence related assault,296,6233,0.1220219,51081,170.5


Observations: 8,060
Variables: 7
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ Offence            <fct> "Murder *", "Attempted murder", "Murder accessor...
$ Crim_Count         <int> 2, 1, 0, 0, 284, 296, 25, 47, 68, 2, 12, 3, 6, 0...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...
$ Pop_Dens_km2       <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, 170.5,...


In [63]:
out_filename = "joined_crime_popdensity.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `seifa`

In [64]:
head(input_data$crime)
head(input_data$seifa)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


X1,LGA,Value
1,albury,964
2,armidale dumaresq,954
3,ashfield,962
4,auburn,931
5,ballina,986
6,balranald,969


In [65]:
joined <- input_data$crime %>%
    left_join(input_data$seifa %>% select(-X1)) %>%
    rename(seifa = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,seifa
albury,Murder *,2,6233,0.1220219,51081,964
albury,Attempted murder,1,6233,0.1220219,51081,964
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081,964
albury,Manslaughter *,0,6233,0.1220219,51081,964
albury,Domestic violence related assault,284,6233,0.1220219,51081,964
albury,Non-domestic violence related assault,296,6233,0.1220219,51081,964


Observations: 8,122
Variables: 7
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ Offence            <fct> "Murder *", "Attempted murder", "Murder accessor...
$ Crim_Count         <int> 2, 1, 0, 0, 284, 296, 25, 47, 68, 2, 12, 3, 6, 0...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...
$ seifa              <int> 964, 964, 964, 964, 964, 964, 964, 964, 964, 964...


In [66]:
out_filename = "joined_crime_seifa.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `transport`

In [67]:
head(input_data$crime)
head(input_data$transport)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


LGA,route_type,num_service
albury,106,131
albury,204,14
albury,700,1624
armidale regional,106,62
armidale regional,204,10
armidale regional,700,113


In [68]:
joined <- input_data$crime %>%
    left_join(input_data$transport)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total,route_type,num_service
albury,Murder *,2,6233,0.1220219,51081,106,131
albury,Murder *,2,6233,0.1220219,51081,204,14
albury,Murder *,2,6233,0.1220219,51081,700,1624
albury,Attempted murder,1,6233,0.1220219,51081,106,131
albury,Attempted murder,1,6233,0.1220219,51081,204,14
albury,Attempted murder,1,6233,0.1220219,51081,700,1624


Observations: 22,134
Variables: 8
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ Offence            <fct> "Murder *", "Murder *", "Murder *", "Attempted m...
$ Crim_Count         <int> 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 284, 284, 28...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...
$ route_type         <fct> 106, 204, 700, 106, 204, 700, 106, 204, 700, 106...
$ num_service        <dbl> 131, 14, 1624, 131, 14, 1624, 131, 14, 1624, 131...


In [69]:
out_filename = "joined_crime_transport.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Create Aggregated Data Set

In order to have all the variables in one data set, eat data set has to be aggregated at the LGA level. This means losing information on `age` and `sex` etc.

In [70]:
map(input_data, head)

$age
# A tibble: 6 x 5
# Groups:   LGA [6]
  Sex     Age     LGA                          Value pop_total
  <fct>   <fct>   <chr>                        <int>     <int>
1 Females 95 - 99 lismore                         65     43140
2 Females 95 - 99 queanbeyan-palerang regional    33     56023
3 Females 95 - 99 mid-coast                      173     90301
4 Females 95 - 99 glen innes severn               16      8828
5 Females 95 - 99 lockhart                        11      3135
6 Females 95 - 99 lane cove                       49     36053

$country_of_birth
# A tibble: 6 x 7
  birthplace_of_pe~ census_year lga_2016 LGA   value yarrp_2016 year_of_arrival~
  <fct>                   <dbl>    <dbl> <chr> <dbl> <chr>      <fct>           
1 New Zealand              2016    10050 albu~     7 2          Arrived 1946 - ~
2 New Zealand              2016    10050 albu~    22 3          Arrived 1956 - ~
3 New Zealand              2016    10050 albu~    63 4          Arrived 1966 - ~
4 New Zealand              2016    10050 albu~   113 5          Arrived 1976 - ~
5 New Zealand              2016    10050 albu~    92 6          Arrived 1986 - ~
6 New Zealand              2016    10050 albu~    87 7          Arrived 1996 - ~

$crime
# A tibble: 6 x 6
  LGA    Offence          Crim_Count crime_tot_per_l~ crime_rate_per_~ pop_total
  <chr>  <fct>                 <int>            <int>            <dbl>     <int>
1 albury Murder *                  2             6233            0.122     51081
2 albury Attempted murder          1             6233            0.122     51081
3 albury Murder accessor~          0             6233            0.122     51081
4 albury Manslaughter *            0             6233            0.122     51081
5 albury Domestic violen~        284             6233            0.122     51081
6 albury Non-domestic vi~        296             6233            0.122     51081

$industry_employment
# A tibble: 6 x 7
  Age    Sex    `Industry of Employment` LGA_2016 LGA        `Census year` Value
  <fct>  <fct>  <fct>                       <dbl> <chr>              <dbl> <int>
1 15 - ~ Males  Agriculture, Forestry a~    10050 albury              2016     6
2 15 - ~ Femal~ Agriculture, Forestry a~    10050 albury              2016     4
3 15 - ~ Perso~ Agriculture, Forestry a~    10050 albury              2016     6
4 15 - ~ Males  Agriculture, Forestry a~    10130 armidale ~          2016    34
5 15 - ~ Femal~ Agriculture, Forestry a~    10130 armidale ~          2016     9
6 15 - ~ Perso~ Agriculture, Forestry a~    10130 armidale ~          2016    43

$labour
# A tibble: 6 x 7
  Age     Sex     `Labour force status` LGA_2016 LGA         `Census year` Value
  <fct>   <fct>   <fct>                    <dbl> <chr>               <dbl> <int>
1 15 - 19 Males   Employed, worked ful~    10050 albury               2016   236
2 15 - 19 Females Employed, worked ful~    10050 albury               2016   113
3 15 - 19 Persons Employed, worked ful~    10050 albury               2016   348
4 15 - 19 Males   Employed, worked ful~    10130 armidale r~          2016   105
5 15 - 19 Females Employed, worked ful~    10130 armidale r~          2016    58
6 15 - 19 Persons Employed, worked ful~    10130 armidale r~          2016   160

$pop_dens
# A tibble: 6 x 3
     X1 LGA               Pop_Dens_km2
  <dbl> <chr>                    <dbl>
1     1 albury                   170. 
2     2 armidale regional          3.5
3     3 ballina                   88.7
4     4 balranald                  0.1
5     5 bathurst regional         11.1
6     6 bega valley                5.4

$seifa
# A tibble: 6 x 3
     X1 LGA               Value
  <dbl> <chr>             <int>
1     1 albury              964
2     2 armidale dumaresq   954
3     3 ashfield            962
4     4 auburn              931
5     5 ballina             986
6     6 balranald           969

$transport
# A tibble: 6 x 3
  LGA               route_type num_service
  <chr>             <fct>          

In [71]:
value_cols <- list("Value", "value", "Crim_Count", "Value", "Value", "Pop_Dens_km2", "Value", "num_service")

In [72]:
names(value_cols) <- tbl_names
names(value_cols)

[1] "age"                 "country_of_birth"    "crime"              
[4] "industry_employment" "labour"              "pop_dens"           
[7] "seifa"               "transport"

# Rename Columns and Aggregate

In [73]:
head(input_data$age)

Sex,Age,LGA,Value,pop_total
Females,95 - 99,lismore,65,43140
Females,95 - 99,queanbeyan-palerang regional,33,56023
Females,95 - 99,mid-coast,173,90301
Females,95 - 99,glen innes severn,16,8828
Females,95 - 99,lockhart,11,3135
Females,95 - 99,lane cove,49,36053


In [74]:
age_agg <- input_data$age %>%
    rename(pop = Value)

head(age_agg)
glimpse(age_agg)

Sex,Age,LGA,pop,pop_total
Females,95 - 99,lismore,65,43140
Females,95 - 99,queanbeyan-palerang regional,33,56023
Females,95 - 99,mid-coast,173,90301
Females,95 - 99,glen innes severn,16,8828
Females,95 - 99,lockhart,11,3135
Females,95 - 99,lane cove,49,36053


Observations: 5,808
Variables: 5
Groups: LGA [132]
$ Sex       <fct> Females, Females, Females, Females, Females, Females, Fem...
$ Age       <fct> 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 ...
$ LGA       <chr> "lismore", "queanbeyan-palerang regional", "mid-coast", "...
$ pop       <int> 65, 33, 173, 16, 11, 49, 26, 451, 24, 0, 4, 39, 72, 29, 9...
$ pop_total <int> 43140, 56023, 90301, 8828, 3135, 36053, 19195, 252898, 16...


## Country of Birth

In [75]:
head(input_data$country_of_birth)

birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,albury,87,7,Arrived 1996 - 2005


In [76]:
country_agg <- input_data$country_of_birth %>%
    group_by(LGA, birthplace_of_person) %>%
    summarise(birth_value = sum(value))

head(country_agg)
glimpse(country_agg)

LGA,birthplace_of_person,birth_value
albury,New Zealand,2510
albury,Papua New Guinea,99
albury,Fiji,218
albury,Tonga,24
albury,England,4254
albury,Northern Ireland,64


Observations: 8,508
Variables: 3
Groups: LGA [132]
$ LGA                  <chr> "albury", "albury", "albury", "albury", "albur...
$ birthplace_of_person <fct> "New Zealand", "Papua New Guinea", "Fiji", "To...
$ birth_value          <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325, 25...


## Industry Employment

In [77]:
head(input_data$industry_employment)

Age,Sex,Industry of Employment,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Agriculture, Forestry and Fishing",10050,albury,2016,6
15 - 19,Females,"Agriculture, Forestry and Fishing",10050,albury,2016,4
15 - 19,Persons,"Agriculture, Forestry and Fishing",10050,albury,2016,6
15 - 19,Males,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,34
15 - 19,Females,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,9
15 - 19,Persons,"Agriculture, Forestry and Fishing",10130,armidale regional,2016,43


In [78]:
employment_agg <- input_data$industry_employment %>%
    select(-LGA_2016, -`Census year`) %>%
    rename(emp_value = Value)

head(employment_agg)
glimpse(employment_agg)

Age,Sex,Industry of Employment,LGA,emp_value
15 - 19,Males,"Agriculture, Forestry and Fishing",albury,6
15 - 19,Females,"Agriculture, Forestry and Fishing",albury,4
15 - 19,Persons,"Agriculture, Forestry and Fishing",albury,6
15 - 19,Males,"Agriculture, Forestry and Fishing",armidale regional,34
15 - 19,Females,"Agriculture, Forestry and Fishing",armidale regional,9
15 - 19,Persons,"Agriculture, Forestry and Fishing",armidale regional,43


Observations: 79,200
Variables: 5
$ Age                      <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 1...
$ Sex                      <fct> Males, Females, Persons, Males, Females, P...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ LGA                      <chr> "albury", "albury", "albury", "armidale re...
$ emp_value                <int> 6, 4, 6, 34, 9, 43, 27, 10, 39, 0, 0, 6, 1...


## Labour

In [79]:
head(input_data$labour)

Age,Sex,Labour force status,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Employed, worked full-time",10050,albury,2016,236
15 - 19,Females,"Employed, worked full-time",10050,albury,2016,113
15 - 19,Persons,"Employed, worked full-time",10050,albury,2016,348
15 - 19,Males,"Employed, worked full-time",10130,armidale regional,2016,105
15 - 19,Females,"Employed, worked full-time",10130,armidale regional,2016,58
15 - 19,Persons,"Employed, worked full-time",10130,armidale regional,2016,160


In [80]:
labour_agg <- input_data$labour %>%
    select(-LGA_2016, -`Census year`) %>%
    filter(!grepl("Total", `Labour force status`)) %>%
    rename(labour_value = Value)

head(labour_agg)
glimpse(labour_agg)

Age,Sex,Labour force status,LGA,labour_value
15 - 19,Males,"Employed, worked full-time",albury,236
15 - 19,Females,"Employed, worked full-time",albury,113
15 - 19,Persons,"Employed, worked full-time",albury,348
15 - 19,Males,"Employed, worked full-time",armidale regional,105
15 - 19,Females,"Employed, worked full-time",armidale regional,58
15 - 19,Persons,"Employed, worked full-time",armidale regional,160


Observations: 28,512
Variables: 5
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ LGA                   <chr> "albury", "albury", "albury", "armidale regio...
$ labour_value          <int> 236, 113, 348, 105, 58, 160, 115, 53, 168, 11...


In [81]:
transport_agg <- input_data$transport %>%
    group_by(LGA) %>%
    summarise(num_service = sum(num_service))

head(transport_agg)
glimpse(transport_agg)

LGA,num_service
albury,1769
armidale regional,185
ballina,1353
balranald,4
bathurst regional,2882
bayside,143697


Observations: 128
Variables: 2
$ LGA         <chr> "albury", "armidale regional", "ballina", "balranald", ...
$ num_service <dbl> 1769, 185, 1353, 4, 2882, 143697, 771, 422, 10, 248915,...


# Joining the Aggregated Data

Aggregate the crime per LGA.

In [82]:
head(input_data$crime)

LGA,Offence,Crim_Count,crime_tot_per_lga,crime_rate_per_lga,pop_total
albury,Murder *,2,6233,0.1220219,51081
albury,Attempted murder,1,6233,0.1220219,51081
albury,"Murder accessory, conspiracy",0,6233,0.1220219,51081
albury,Manslaughter *,0,6233,0.1220219,51081
albury,Domestic violence related assault,284,6233,0.1220219,51081
albury,Non-domestic violence related assault,296,6233,0.1220219,51081


In [83]:
joined_agg <- input_data$crime %>%
    group_by(LGA) %>%
    summarise(crime_tot_per_lga = sum(Crim_Count), pop_total = mean(pop_total), crime_rate_per_lga = mean(crime_rate_per_lga))

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,pop_total,crime_rate_per_lga
albury,6233,51081,0.12202189
armidale regional,4028,29429,0.13687179
ballina,2690,41774,0.06439412
balranald,192,2292,0.08376963
bathurst regional,4027,41297,0.09751314
bayside,13024,NA,NA


Observations: 130
Variables: 4
$ LGA                <chr> "albury", "armidale regional", "ballina", "balra...
$ crime_tot_per_lga  <int> 6233, 4028, 2690, 192, 4027, 13024, 2119, 798, 4...
$ pop_total          <dbl> 51081, 29429, 41774, 2292, 41297, NA, 33229, 126...
$ crime_rate_per_lga <dbl> 0.12202189, 0.13687179, 0.06439412, 0.08376963, ...


## Joining `age`

In [84]:
joined_agg %<>%
    select(-pop_total) %>%
    left_join(age_agg, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total
albury,6233,0.1220219,Females,95 - 99,78,51081
albury,6233,0.1220219,Males,95 - 99,21,51081
albury,6233,0.1220219,Females,90 - 94,271,51081
albury,6233,0.1220219,Males,90 - 94,117,51081
albury,6233,0.1220219,Females,85 - 89,484,51081
albury,6233,0.1220219,Males,85 - 89,351,51081


Observations: 5,505
Variables: 7
$ LGA                <chr> "albury", "albury", "albury", "albury", "albury"...
$ crime_tot_per_lga  <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0.12...
$ Sex                <fct> Females, Males, Females, Males, Females, Males, ...
$ Age                <fct> 95 - 99, 95 - 99, 90 - 94, 90 - 94, 85 - 89, 85 ...
$ pop                <int> 78, 21, 271, 117, 484, 351, 734, 501, 924, 746, ...
$ pop_total          <int> 51081, 51081, 51081, 51081, 51081, 51081, 51081,...


## Add Country of Birth

In [85]:
head(country_agg)

LGA,birthplace_of_person,birth_value
albury,New Zealand,2510
albury,Papua New Guinea,99
albury,Fiji,218
albury,Tonga,24
albury,England,4254
albury,Northern Ireland,64


In [86]:
joined_agg %<>%
    left_join(country_agg, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64


Observations: 354,953
Variables: 9
$ LGA                  <chr> "albury", "albury", "albury", "albury", "albur...
$ crime_tot_per_lga    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233...
$ crime_rate_per_lga   <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219, 0....
$ Sex                  <fct> Females, Females, Females, Females, Females, F...
$ Age                  <fct> 95 - 99, 95 - 99, 95 - 99, 95 - 99, 95 - 99, 9...
$ pop                  <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total            <int> 51081, 51081, 51081, 51081, 51081, 51081, 5108...
$ birthplace_of_person <fct> "New Zealand", "Papua New Guinea", "Fiji", "To...
$ birth_value          <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325, 25...


## Add Industry Employment

In [87]:
head(employment_agg)

Age,Sex,Industry of Employment,LGA,emp_value
15 - 19,Males,"Agriculture, Forestry and Fishing",albury,6
15 - 19,Females,"Agriculture, Forestry and Fishing",albury,4
15 - 19,Persons,"Agriculture, Forestry and Fishing",albury,6
15 - 19,Males,"Agriculture, Forestry and Fishing",armidale regional,34
15 - 19,Females,"Agriculture, Forestry and Fishing",armidale regional,9
15 - 19,Persons,"Agriculture, Forestry and Fishing",armidale regional,43


In [88]:
joined_agg %<>%
    left_join(employment_agg, by = c("LGA", "Age", "Sex"))

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `Age` joining factors with different levels, coercing to character vector"Warning message:
"Column `Sex` joining factors with different levels, coercing to character vector"

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA


Observations: 968,045
Variables: 11
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ Sex                      <chr> "Females", "Females", "Females", "Females"...
$ Age                      <chr> "95 - 99", "95 - 99", "95 - 99", "95 - 99"...
$ pop                      <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ birthplace_of_person     <fct> "New Zealand", "Papua New Guinea", "Fiji",...
$ birth_value              <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325...
$ `Industry of Employment` <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ emp_value                <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...


## Add Labour

In [89]:
head(labour_agg)

Age,Sex,Labour force status,LGA,labour_value
15 - 19,Males,"Employed, worked full-time",albury,236
15 - 19,Females,"Employed, worked full-time",albury,113
15 - 19,Persons,"Employed, worked full-time",albury,348
15 - 19,Males,"Employed, worked full-time",armidale regional,105
15 - 19,Females,"Employed, worked full-time",armidale regional,58
15 - 19,Persons,"Employed, worked full-time",armidale regional,160


In [90]:
joined_agg %<>%
    left_join(labour_agg, by = c("LGA", "Age", "Sex"))

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `Age` joining character vector and factor, coercing into character vector"Warning message:
"Column `Sex` joining character vector and factor, coercing into character vector"

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA,NA,NA
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA,NA,NA


Observations: 5,485,565
Variables: 13
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ Sex                      <chr> "Females", "Females", "Females", "Females"...
$ Age                      <chr> "95 - 99", "95 - 99", "95 - 99", "95 - 99"...
$ pop                      <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ birthplace_of_person     <fct> "New Zealand", "Papua New Guinea", "Fiji",...
$ birth_value              <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325...
$ `Industry of Employment` <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ emp_value                <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ `Labour force status`    <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ labour_value

## Add Population Density

In [91]:
head(input_data$pop_dens)

X1,LGA,Pop_Dens_km2
1,albury,170.5
2,armidale regional,3.5
3,ballina,88.7
4,balranald,0.1
5,bathurst regional,11.1
6,bega valley,5.4


In [92]:
joined_agg %<>%
    left_join(input_data$pop_dens %>% select(-X1), by = "LGA")

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA,NA,NA,170.5
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA,NA,NA,170.5
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA,NA,NA,170.5
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA,NA,NA,170.5
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA,NA,NA,170.5
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA,NA,NA,170.5


Observations: 5,485,565
Variables: 14
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ Sex                      <chr> "Females", "Females", "Females", "Females"...
$ Age                      <chr> "95 - 99", "95 - 99", "95 - 99", "95 - 99"...
$ pop                      <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ birthplace_of_person     <fct> "New Zealand", "Papua New Guinea", "Fiji",...
$ birth_value              <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325...
$ `Industry of Employment` <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ emp_value                <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ `Labour force status`    <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ labour_value

## Add SEIFA

In [93]:
head(input_data$seifa)

X1,LGA,Value
1,albury,964
2,armidale dumaresq,954
3,ashfield,962
4,auburn,931
5,ballina,986
6,balranald,969


In [94]:
joined_agg %<>%
    left_join(input_data$seifa %>% select(-X1) %>% rename(seifa_value = Value), by = "LGA")

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA,NA,NA,170.5,964
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA,NA,NA,170.5,964
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA,NA,NA,170.5,964
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA,NA,NA,170.5,964
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA,NA,NA,170.5,964
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA,NA,NA,170.5,964


Observations: 5,588,245
Variables: 15
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ Sex                      <chr> "Females", "Females", "Females", "Females"...
$ Age                      <chr> "95 - 99", "95 - 99", "95 - 99", "95 - 99"...
$ pop                      <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ birthplace_of_person     <fct> "New Zealand", "Papua New Guinea", "Fiji",...
$ birth_value              <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325...
$ `Industry of Employment` <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ emp_value                <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ `Labour force status`    <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ labour_value

## Add Transport

In [95]:
head(transport_agg)

LGA,num_service
albury,1769
armidale regional,185
ballina,1353
balranald,4
bathurst regional,2882
bayside,143697


In [96]:
joined_agg %<>%
    left_join(transport_agg, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value,num_service
albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA,NA,NA,170.5,964,1769
albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA,NA,NA,170.5,964,1769
albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA,NA,NA,170.5,964,1769
albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA,NA,NA,170.5,964,1769
albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA,NA,NA,170.5,964,1769
albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA,NA,NA,170.5,964,1769


Observations: 5,588,245
Variables: 16
$ LGA                      <chr> "albury", "albury", "albury", "albury", "a...
$ crime_tot_per_lga        <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate_per_lga       <dbl> 0.1220219, 0.1220219, 0.1220219, 0.1220219...
$ Sex                      <chr> "Females", "Females", "Females", "Females"...
$ Age                      <chr> "95 - 99", "95 - 99", "95 - 99", "95 - 99"...
$ pop                      <int> 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78...
$ pop_total                <int> 51081, 51081, 51081, 51081, 51081, 51081, ...
$ birthplace_of_person     <fct> "New Zealand", "Papua New Guinea", "Fiji",...
$ birth_value              <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325...
$ `Industry of Employment` <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ emp_value                <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ `Labour force status`    <fct> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA...
$ labour_value

In [98]:
write.csv(joined_agg, paste(output_path, "unified3.csv", sep = "/"), row.names = FALSE)

In [121]:
save.image()

In [1]:
head(joined_agg)

LGA,crime_tot_per_lga,crime_rate_per_lga,Sex,Age,pop,pop_total,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value,num_service
Albury,6233,0.1220219,Females,95 - 99,78,51081,New Zealand,2510,NA,NA,NA,NA,170.5,964,1769
Albury,6233,0.1220219,Females,95 - 99,78,51081,Papua New Guinea,99,NA,NA,NA,NA,170.5,964,1769
Albury,6233,0.1220219,Females,95 - 99,78,51081,Fiji,218,NA,NA,NA,NA,170.5,964,1769
Albury,6233,0.1220219,Females,95 - 99,78,51081,Tonga,24,NA,NA,NA,NA,170.5,964,1769
Albury,6233,0.1220219,Females,95 - 99,78,51081,England,4254,NA,NA,NA,NA,170.5,964,1769
Albury,6233,0.1220219,Females,95 - 99,78,51081,Northern Ireland,64,NA,NA,NA,NA,170.5,964,1769


In [97]:
joined_agg %>%
    filter(is.na(num_service)) %>%
    select(LGA) %>%
    unique

LGA
broken hill
carrathool
cootamundra-gundagai
glen innes severn
greater hume shire
lord howe island
unincorporated far west
upper hunter shire
warrumbungle shire


In [10]:
transport_agg %>%
    select(LGA) %>%
    filter(grepl(" ", LGA))

LGA
Armidale Regional
Bathurst Regional
Bega Valley
Blue Mountains
Canada Bay
Central Coast
Central Darling
Clarence Valley
Coffs Harbour
Cootamundra-Gundagai Regional


In [40]:
head(input_data$crime)

LGA,Offence,Crim_Count,crime_tot_per_lga,pop_total,crime_rate_per_lga
albury,Murder *,2,6233,51081,0.1220219
albury,Attempted murder,1,6233,51081,0.1220219
albury,"Murder accessory, conspiracy",0,6233,51081,0.1220219
albury,Manslaughter *,0,6233,51081,0.1220219
albury,Domestic violence related assault,284,6233,51081,0.1220219
albury,Non-domestic violence related assault,296,6233,51081,0.1220219
